In [1]:
import os
import gc
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.stats import rankdata
import textstat
from tqdm import tqdm
from typing import Dict, NamedTuple
import scml
import mylib

In [2]:
class Conf(NamedTuple):
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pretrained_dir: str = "pretrained/"
    dtfy_model_max_length: int = 512
    dtfy_batch_size: int = 64
    dtfy_models: Dict[str, str] = {
        "dto_": f"{pretrained_dir}unitaryai/detoxify/toxic_original-c1212f89.ckpt",
        "dtu_": f"{pretrained_dir}unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt",
        "dtm_": f"{pretrained_dir}unitaryai/detoxify/multilingual_debiased-0b549669.ckpt"
    }
    dtfy_configs: Dict[str, str] = {
        "dto_": f"{pretrained_dir}bert-base-uncased",
        "dtu_": f"{pretrained_dir}roberta-base",
        "dtm_": f"{pretrained_dir}xlm-roberta-base"
    }
    hatebert_model_max_length: int = 512
    hatebert_batch_size: int = 128
    hatebert_models: Dict[str, str] = {
        "hb_bert_off": f"{pretrained_dir}/hatebert/bert-offenseval",
        "hb_bert_abu" : f"{pretrained_dir}/hatebert/bert-abuseval",
        "hb_hatebert_off": f"{pretrained_dir}/hatebert/hatebert-offenseval",
        "hb_hatebert_abu" : f"{pretrained_dir}/hatebert/hatebert-abuseval",
    }
    em_max_seq_length: int = 128
    em_batch_size: int = 1000
    em_models: Dict[str, str] = {
        "paraphrase-MiniLM-L6-v2": f"{pretrained_dir}sentence-transformers/paraphrase-MiniLM-L6-v2"
    }
        
        
conf = Conf()
print(conf)
if conf.device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Conf(device=device(type='cuda'), pretrained_dir='pretrained/', dtfy_model_max_length=512, dtfy_batch_size=64, dtfy_models={'dto_': 'pretrained/unitaryai/detoxify/toxic_original-c1212f89.ckpt', 'dtu_': 'pretrained/unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt', 'dtm_': 'pretrained/unitaryai/detoxify/multilingual_debiased-0b549669.ckpt'}, dtfy_configs={'dto_': 'pretrained/bert-base-uncased', 'dtu_': 'pretrained/roberta-base', 'dtm_': 'pretrained/xlm-roberta-base'}, hatebert_model_max_length=512, hatebert_batch_size=128, hatebert_models={'hb_bert_off': 'pretrained//hatebert/bert-offenseval', 'hb_bert_abu': 'pretrained//hatebert/bert-abuseval', 'hb_hatebert_off': 'pretrained//hatebert/hatebert-offenseval', 'hb_hatebert_abu': 'pretrained//hatebert/hatebert-abuseval'}, em_max_seq_length=128, em_batch_size=1000, em_models={'paraphrase-MiniLM-L6-v2': 'pretrained/sentence-transformers/paraphrase-MiniLM-L6-v2'})
NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [4]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
Wall time: 221 ms


In [5]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text

In [6]:
def preprocess(row) -> str:
    return mylib.preprocess(row["text"])


col = "ptext"
df[col] = df.progress_apply(preprocess, axis=1)

100%|███████████████████████████████████████| 14251/14251 [00:10<00:00, 1320.01it/s]


# Character level features

In [7]:
%%time
col = "length"
df[col] = df["ptext"].str.len()
df[col] = df[col].astype(np.int16)
char_fs = [col]

Wall time: 14.2 ms


In [8]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row["ptext"])


def letter_frac(row) -> float:
    return mylib.letter_frac(row["ptext"])


def space_frac(row) -> float:
    return mylib.space_frac(row["ptext"])


def punc_frac(row) -> float:
    return mylib.punc_frac(row["ptext"])


def upper_frac(row) -> float:
    return mylib.upper_frac(row["ptext"])

In [9]:
col = "digit_frac"
df[col] = df.progress_apply(digit_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 23844.72it/s]


In [10]:
col = "letter_frac"
df[col] = df.progress_apply(letter_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 22140.38it/s]


In [11]:
col = "space_frac"
df[col] = df.progress_apply(space_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 24146.27it/s]


In [12]:
col = "punc_frac"
df[col] = df.progress_apply(punc_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 20784.74it/s]


In [13]:
col = "upper_frac"
df[col] = df.progress_apply(upper_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 22922.63it/s]


In [14]:
print(char_fs)

['length', 'digit_frac', 'letter_frac', 'space_frac', 'punc_frac', 'upper_frac']


# Textstat features

In [15]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row["ptext"])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row["ptext"])


def sentence_count(row) -> int:
    return textstat.sentence_count(row["ptext"])


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row["ptext"])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row["ptext"])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row["ptext"])


def smog_index(row) -> float:
    return textstat.smog_index(row["ptext"])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row["ptext"])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row["ptext"])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row["ptext"])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row["ptext"])

In [16]:
col = "flesch_reading_ease"
df[col] = df.progress_apply(flesch_reading_ease, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs = []
textstat_fs.append(col)

100%|███████████████████████████████████████| 14251/14251 [00:03<00:00, 3788.31it/s]


In [17]:
col = "flesch_kincaid_grade"
df[col] = df.progress_apply(flesch_kincaid_grade, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|███████████████████████████████████████| 14251/14251 [00:01<00:00, 7280.60it/s]


In [18]:
col = "syllable_count"
df[col] = df.progress_apply(syllable_count, axis=1)
df[col] = df[col].astype(np.int16)
textstat_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:01<00:00, 10798.48it/s]


In [19]:
col = "lexicon_count"
df[col] = df.progress_apply(lexicon_count, axis=1)
df[col] = df[col].astype(np.int16)
textstat_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 50896.30it/s]


In [20]:
col = "sentence_count"
df[col] = df.progress_apply(sentence_count, axis=1)
df[col] = df[col].astype(np.int16)
textstat_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 25492.49it/s]


In [21]:
col = "gunning_fog"
df[col] = df.progress_apply(gunning_fog, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|███████████████████████████████████████| 14251/14251 [00:02<00:00, 5249.99it/s]


In [22]:
col = "smog_index"
df[col] = df.progress_apply(smog_index, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|███████████████████████████████████████| 14251/14251 [00:01<00:00, 7972.53it/s]


In [23]:
col = "automated_readability_index"
df[col] = df.progress_apply(automated_readability_index, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 18884.07it/s]


In [24]:
col = "coleman_liau_index"
df[col] = df.progress_apply(coleman_liau_index, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 16477.09it/s]


In [25]:
col = "linsear_write_formula"
df[col] = df.progress_apply(linsear_write_formula, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|███████████████████████████████████████| 14251/14251 [00:01<00:00, 8864.09it/s]


In [26]:
col = "dale_chall_readability_score"
df[col] = df.progress_apply(dale_chall_readability_score, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|███████████████████████████████████████| 14251/14251 [00:02<00:00, 6099.78it/s]


In [27]:
print(textstat_fs)

['flesch_reading_ease', 'flesch_kincaid_grade', 'syllable_count', 'lexicon_count', 'sentence_count', 'gunning_fog', 'smog_index', 'automated_readability_index', 'coleman_liau_index', 'linsear_write_formula', 'dale_chall_readability_score']


# HateBert labels

In [28]:
sentences = list(df["ptext"])
# all Hatebert models use the same tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    conf.hatebert_models["hb_hatebert_off"], 
    model_max_length=conf.hatebert_model_max_length
)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained//hatebert/hatebert-offenseval', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']


In [29]:
%%time
x = tokenizer(sentences, truncation=True, padding="max_length")
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=14251
Wall time: 4.41 s


In [30]:
batches = torch.utils.data.DataLoader(mylib.Dataset(x), batch_size=conf.hatebert_batch_size, shuffle=False)
for col, model_dir in conf.hatebert_models.items():    
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    model.to(conf.device)
    logits = None
    with torch.no_grad():
        for batch in tqdm(batches):
            for k, v in batch.items():
                batch[k] = v.to(conf.device)
            outputs = model(**batch)
            tmp = outputs.logits.detach().cpu()
            if logits is None:
                logits = tmp
            else:
                logits = torch.cat((logits, tmp), 0)
    logits = torch.nn.functional.softmax(logits, dim=1)
    print(f"{logits.size()}\nlogits[:10]={logits[:10]}")
    df[col] = logits[:,1]
    df[col] = df[col].astype(np.float32)

100%|█████████████████████████████████████████████| 112/112 [10:22<00:00,  5.55s/it]


torch.Size([14251, 2])
logits[:10]=tensor([[0.0259, 0.9741],
        [0.0444, 0.9556],
        [0.4875, 0.5125],
        [0.2584, 0.7416],
        [0.5296, 0.4704],
        [0.0798, 0.9202],
        [0.4345, 0.5655],
        [0.9865, 0.0135],
        [0.2440, 0.7560],
        [0.0346, 0.9654]])


100%|█████████████████████████████████████████████| 112/112 [10:19<00:00,  5.53s/it]


torch.Size([14251, 2])
logits[:10]=tensor([[0.0268, 0.9732],
        [0.0319, 0.9681],
        [0.3978, 0.6022],
        [0.2262, 0.7738],
        [0.6494, 0.3506],
        [0.4385, 0.5615],
        [0.5164, 0.4836],
        [0.9960, 0.0040],
        [0.4188, 0.5812],
        [0.0309, 0.9691]])


100%|█████████████████████████████████████████████| 112/112 [10:19<00:00,  5.53s/it]


torch.Size([14251, 2])
logits[:10]=tensor([[0.0353, 0.9647],
        [0.0539, 0.9461],
        [0.4675, 0.5325],
        [0.4235, 0.5765],
        [0.6997, 0.3003],
        [0.1006, 0.8994],
        [0.5238, 0.4762],
        [0.9628, 0.0372],
        [0.2800, 0.7200],
        [0.0372, 0.9628]])


100%|█████████████████████████████████████████████| 112/112 [10:19<00:00,  5.53s/it]

torch.Size([14251, 2])
logits[:10]=tensor([[0.0273, 0.9727],
        [0.0413, 0.9587],
        [0.8282, 0.1718],
        [0.4918, 0.5082],
        [0.8416, 0.1584],
        [0.9625, 0.0375],
        [0.8139, 0.1861],
        [0.9878, 0.0122],
        [0.3514, 0.6486],
        [0.0730, 0.9270]])


# Detoxify labels

In [31]:
gc.collect()
dtfy_fs = []
for prefix, checkpoint in tqdm(conf.dtfy_models.items()):
    res = mylib.detoxify_labels(
        sentences,
        checkpoint=checkpoint,
        config_dir=conf.dtfy_configs[prefix],
        model_max_length=conf.dtfy_model_max_length,
        device=conf.device,
        batch_size=conf.dtfy_batch_size
    )
    for k, v in res.items():
        col = prefix + k
        df[col] = v
        df[col] = df[col].astype(np.float32)
        dtfy_fs.append(col)
    gc.collect()

100%|████████████████████████████████████████████████| 3/3 [30:23<00:00, 607.98s/it]


In [32]:
print(dtfy_fs)

['dto_toxicity', 'dto_severe_toxicity', 'dto_obscene', 'dto_threat', 'dto_insult', 'dto_identity_attack', 'dtu_toxicity', 'dtu_severe_toxicity', 'dtu_obscene', 'dtu_identity_attack', 'dtu_insult', 'dtu_threat', 'dtu_sexual_explicit', 'dtm_toxicity', 'dtm_severe_toxicity', 'dtm_obscene', 'dtm_identity_attack', 'dtm_insult', 'dtm_threat', 'dtm_sexual_explicit']


# Embeddings

In [33]:
model = SentenceTransformer(conf.em_models["paraphrase-MiniLM-L6-v2"], device=conf.device)
model.max_seq_length = conf.em_max_seq_length
em = model.encode(sentences=sentences, batch_size=conf.em_batch_size, show_progress_bar=True, convert_to_numpy=True)
print(f"em.shape={em.shape}")

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

em.shape=(14251, 384)


In [34]:
%%time
em_size = em.shape[1]
em_cols = [f"zz{i:04d}" for i in range(em_size)]
df[em_cols] = em
df[em_cols] = df[em_cols].astype(np.float32)
del sentences

s:\dev\seahrh\kaggle-jigsaw-toxic-severity-rating\env\lib\site-packages\pandas\core\frame.py:3673: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Wall time: 270 ms


# Review data

In [35]:
cols = char_fs + textstat_fs + dtfy_fs + list(conf.hatebert_models.keys())
df[cols].describe(percentiles=percentiles)

,length,digit_frac,letter_frac,space_frac,punc_frac,upper_frac,flesch_reading_ease,flesch_kincaid_grade,syllable_count,lexicon_count,sentence_count,gunning_fog,smog_index,automated_readability_index,coleman_liau_index,linsear_write_formula,dale_chall_readability_score,dto_toxicity,dto_severe_toxicity,dto_obscene,dto_threat,dto_insult,dto_identity_attack,dtu_toxicity,dtu_severe_toxicity,dtu_obscene,dtu_identity_attack,dtu_insult,dtu_threat,dtu_sexual_explicit,dtm_toxicity,dtm_severe_toxicity,dtm_obscene,dtm_identity_attack,dtm_insult,dtm_threat,dtm_sexual_explicit,hb_bert_off,hb_bert_abu,hb_hatebert_off,hb_hatebert_abu
count,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000
mean,408.604238,0.009755,0.772961,0.173892,0.043391,0.086143,63.909290,9.625760,100.923233,71.034945,3.974388,11.228957,4.309606,13.042587,9.093494,9.910800,9.519163,0.458109,0.067606,0.294862,0.028744,0.277615,0.061937,0.533886,0.046184,0.297274,0.061978,0.305610,0.018006,0.098901,0.509862,0.062107,0.259716,0.056330,0.269497,0.026773,0.135149,0.636137,0.498080,0.603282,0.458325
std,688.809292,0.032184,0.056109,0.024900,0.043562,0.176861,315.695831,48.136086,169.861304,117.409068,8.581545,21.454451,5.286682,90.704788,97.974510,9.309810,3.388893,0.427506,0.155756,0.393250,0.123391,0.368762,0.177876,0.398096,0.120675,0.390690,0.161861,0.360564,0.095459,0.217916,0.413721,0.158119,0.371604,0.162481,0.351557,0.111058,0.273091,0.346537,0.399806,0.331455,0.380057
min,8.000000,0.000000,0.004427,0.000403,0.000000,0.000000,-36681.820312,-3.100000,2.000000,2.000000,1.000000,0.800000,0.000000,-9.300000,-14.150000,0.000000,0.100000,0.000535,0.000079,0.000149,0.000085,0.000164,0.000127,0.000352,0.000001,0.000017,0.000060,0.000061,0.000016,0.000010,0.000126,0.000011,0.000052,0.000071,0.000128,0.000017,0.000013,0.009207,0.002739,0.008755,0.005502
1%,22.000000,0.000000,0.549739,0.094467,0.000000,0.000000,-69.965000,-1.900000,5.000000,4.000000,1.000000,1.600000,0.000000,-2.900000,-2.950000,1.000000,1.105000,0.000662,0.000087,0.000165,0.000099,0.000175,0.000136,0.001290,0.000003,0.000073,0.000136,0.000104,0.000034,0.000036,0.000405,0.000016,0.000085,0.000108,0.000181,0.000027,0.000018,0.018364,0.004938,0.025118,0.010837
5%,31.000000,0.000000,0.683673,0.131579,0.000000,0.000000,30.200001,0.500000,8.000000,6.000000,1.000000,2.400000,0.000000,0.300000,0.450000,2.000000,6.340000,0.000958,0.000093,0.000179,0.000109,0.000186,0.000145,0.005784,0.000007,0.000271,0.000345,0.000297,0.000067,0.000100,0.001091,0.000024,0.000150,0.000175,0.000374,0.000038,0.000025,0.042573,0.009142,0.056917,0.017714
10%,44.000000,0.000000,0.722973,0.145038,0.012579,0.008547,43.560001,1.900000,11.000000,8.000000,1.000000,3.200000,0.000000,2.000000,2.370000,2.500000,6.930000,0.001849,0.000100,0.000211,0.000119,0.000219,0.000164,0.016431,0.000013,0.000534,0.000588,0.000892,0.000100,0.000186,0.003285,0.000036,0.000332,0.000293,0.000899,0.000059,0.000039,0.082740,0.015286,0.098816,0.025853
20%,69.000000,0.000000,0.750000,0.158537,0.020170,0.016471,55.610001,3.500000,17.000000,12.000000,1.000000,5.200000,0.000000,4.100000,4.290000,4.000000,7.640000,0.009245,0.000114,0.000473,0.000164,0.000519,0.000260,0.060106,0.000030,0.001509,0.001155,0.003852,0.000177,0.000442,0.021863,0.000106,0.001394,0.000720,0.003930,0.000166,0.000138,0.204838,0.038415,0.213655,0.049308
30%,99.000000,0.000000,0.765060,0.166031,0.025000,0.021622,62.680000,4.600000,24.000000,18.000000,1.000000,6.730000,0.000000,5.600000,5.510000,5.000000,8.140000,0.029829,0.000154,0.

In [36]:
cols = ["text"] + cols + em_cols
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 426 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    text                          14251 non-null  object 
 1    length                        14251 non-null  int16  
 2    digit_frac                    14251 non-null  float32
 3    letter_frac                   14251 non-null  float32
 4    space_frac                    14251 non-null  float32
 5    punc_frac                     14251 non-null  float32
 6    upper_frac                    14251 non-null  float32
 7    flesch_reading_ease           14251 non-null  float32
 8    flesch_kincaid_grade          14251 non-null  float32
 9    syllable_count                14251 non-null  int16  
 10   lexicon_count                 14251 non-null  int16  
 11   sentence_count                14251 non-null  int16  
 12   gunning_fog                   14251 non-null

In [37]:
%%time
df[cols].to_parquet("output/val.parquet", index=False)

Wall time: 496 ms
